In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [3]:
%cd '/content/drive/My Drive/intern_icomm_project_demo'
! ls

/content/drive/My Drive/intern_icomm_project_demo
data		  K_mean_nlp.ipynb	   mnist.ipynb		     x28.txt.1
data_all_nlp.csv  linear_regression.ipynb  vietnamese-stopwords.txt
data.zip	  log.csv		   x28.txt


In [4]:
def warn(*args, **kwargs):
  pass
import warnings
warnings.warn = warn

In [7]:
import keras
from keras.models import Sequential
from keras.utils import np_utils
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Activation, Flatten, Dropout, BatchNormalization
from keras.layers import Conv2D, MaxPooling2D
from keras.datasets import mnist
from keras import regularizers
from keras.callbacks import LearningRateScheduler
import numpy as np

In [6]:
def lr_schedule(epoch):
    lrate = 0.001
    if epoch > 75:
        lrate = 0.0005
    if epoch > 100:
        lrate = 0.0003
    return lrate

In [15]:

# The data, split between train and test sets:
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_val = x_train[50000:60000,:,:]
y_val = y_train[50000:60000]

x_train = x_train[:50000,:,:]
y_train = y_train[:50000]

print('x_train shape:', x_train.shape)
print('x_train shape:', y_train.shape)
print(x_train.shape[0], 'train samples')
print(x_val.shape[0], 'val samples')
print(x_test.shape[0], 'test samples')




x_train shape: (50000, 28, 28)
x_train shape: (50000,)
50000 train samples
10000 val samples
10000 test samples


In [17]:
#z-score
mean = np.mean(x_train,axis=(0,1,2))
std = np.std(x_train,axis=(0,1,2))
x_train = (x_train-mean)/(std+1e-7)
x_val = (x_val-mean)/(std+1e-7)
x_test = (x_test-mean)/(std+1e-7)

num_classes = 10
# Convert class vectors to binary class matrices.
y_train = keras.utils.to_categorical(y_train, num_classes= None, dtype = "float32")
y_test = keras.utils.to_categorical(y_test, num_classes=None, dtype = "float32")
y_val = keras.utils.to_categorical(y_val, num_classes=None, dtype = "float32")

In [35]:
x_train = x_train.reshape(50000,28,28,1)
x_test = x_test.reshape(10000,28,28,1)
x_val = x_val.reshape(10000,28,28,1)

In [36]:
weight_decay = 1e-4
model = Sequential()
model.add(Conv2D(32, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay), input_shape=x_train.shape[1:]))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Conv2D(32, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

model.add(Conv2D(64, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Conv2D(64, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.3))

model.add(Conv2D(128, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Conv2D(128, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Flatten())
model.add(Dense(num_classes, activation='softmax'))

model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_15 (Conv2D)           (None, 28, 28, 32)        320       
_________________________________________________________________
activation_12 (Activation)   (None, 28, 28, 32)        0         
_________________________________________________________________
batch_normalization_12 (Batc (None, 28, 28, 32)        128       
_________________________________________________________________
conv2d_16 (Conv2D)           (None, 28, 28, 32)        9248      
_________________________________________________________________
activation_13 (Activation)   (None, 28, 28, 32)        0         
_________________________________________________________________
batch_normalization_13 (Batc (None, 28, 28, 32)        128       
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 14, 14, 32)       

In [37]:
#data augmentation
datagen = ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    )
datagen.fit(x_train)

In [40]:
#training
batch_size = 64

opt_rms = keras.optimizers.RMSprop(lr=0.001,decay=1e-6)
model.compile(loss='categorical_crossentropy', optimizer=opt_rms, metrics=['accuracy'])
model.fit_generator(datagen.flow(x_train, y_train, batch_size=batch_size),\
                    steps_per_epoch=x_train.shape[0] // batch_size,epochs=25,\
                    verbose=1,validation_data=(x_test,y_test),callbacks=[LearningRateScheduler(lr_schedule)])
#save to disk
model_json = model.to_json()
with open('model.json', 'w') as json_file:
    json_file.write(model_json)
model.save_weights('model.h5') 



Epoch 1/25
781/781 [==============================] - 22s 28ms/step - loss: 0.1220 - accuracy: 0.9789 - val_loss: 0.0908 - val_accuracy: 0.9880
Epoch 2/25
781/781 [==============================] - 22s 28ms/step - loss: 0.1212 - accuracy: 0.9793 - val_loss: 0.0915 - val_accuracy: 0.9881
Epoch 3/25
781/781 [==============================] - 22s 28ms/step - loss: 0.1208 - accuracy: 0.9791 - val_loss: 0.0877 - val_accuracy: 0.9884
Epoch 4/25
781/781 [==============================] - 22s 29ms/step - loss: 0.1184 - accuracy: 0.9801 - val_loss: 0.1075 - val_accuracy: 0.9819
Epoch 5/25
781/781 [==============================] - 22s 29ms/step - loss: 0.1161 - accuracy: 0.9798 - val_loss: 0.0944 - val_accuracy: 0.9876
Epoch 6/25
781/781 [==============================] - 22s 28ms/step - loss: 0.1165 - accuracy: 0.9795 - val_loss: 0.0898 - val_accuracy: 0.9879
Epoch 7/25
781/781 [==============================] - 22s 28ms/step - loss: 0.1155 - accuracy: 0.9798 - val_loss: 0.0913 - val_accuracy:

In [41]:
#testing
scores = model.evaluate(x_val, y_val, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

313/313 [==============================] - 1s 4ms/step - loss: 0.0908 - accuracy: 0.9875
Test loss: 0.0908169224858284
Test accuracy: 0.987500011920929


In [54]:
y_pred = model.predict(x_val)

In [56]:
y_pre = np.argmax(y_pred, axis=1)

In [57]:
y_pre

array([3, 8, 6, ..., 5, 6, 8])

In [58]:
y_va = np.argmax(y_val, axis=1)

In [59]:
y_va

array([3, 8, 6, ..., 5, 6, 8])

In [45]:
from sklearn import metrics 

In [63]:
print("Accuracy: ", 100*metrics.accuracy_score(y_va, y_pre))

print("Precision: ", 100*metrics.precision_score(y_va, y_pre, average='macro'))

print("Precision: ", 100*metrics.precision_score(y_va, y_pre, average='micro'))
print("Recall (macro): ", 100*metrics.recall_score(y_va, y_pre, average='macro'))


print("Recall(micro): ", 100*metrics.recall_score(y_va, y_pre, average='micro'))

print("F1-scores(macro): ", 100*metrics.f1_score(y_va, y_pre, average='macro'))

print("F1-scores(micro): ", 100*metrics.f1_score(y_va, y_pre, average='micro'))

Accuracy:  98.75
Precision:  98.71594877896766
Precision:  98.75
Recall (macro):  98.73899988862152
Recall(micro):  98.75
F1-scores(macro):  98.72489837616023
F1-scores(micro):  98.75


In [ ]:
# Code load model to test
from keras.models import load_model
model = load_model('model.h5')
scores = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])